# A Dummy Model

## Preamble

In [7]:
import pandas

In [8]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

## Data Preparation

In [9]:
movies = catalog.load("movies")

2021-03-16 10:22:04,663 - kedro.io.data_catalog - INFO - Loading data from `movies` (ParquetDataSet)...


In [10]:
#movies = movies.set_index("id")

In [11]:
movies["genre_names"] = movies["genres"].apply(lambda ls: [d["name"] for d in ls])

In [12]:
movies = movies[movies["genre_names"].map(lambda d: len(d)) > 0] # drop  movies without genres

In [13]:
genre_labels = pandas.get_dummies(
    movies["genre_names"].apply(pandas.Series).stack(dropna=False),
    prefix="Genre"
).sum(level=0)

In [14]:
posters = movies[["original_title", "poster_path"]]

In [15]:
genre_labels

,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,Genre_Family,Genre_Fantasy,Genre_History,Genre_Horror,Genre_Music,Genre_Mystery,Genre_Romance,Genre_Science Fiction,Genre_TV Movie,Genre_Thriller,Genre_War,Genre_Western
id,,,,,,,,,,,,,,,,,,,
2,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
19996,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [16]:
movies[~movies.index.isin(genre_labels.index)]

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,genre_names
id,,,,,,,,,,,,,,,,,,,,,


## Metrics

In [17]:
from sklearn.metrics import f1_score

In [18]:
from sklearn.model_selection import cross_val_score

In [19]:
from sklearn.metrics import make_scorer

## Dummy Classifier

In [20]:
from sklearn.dummy import DummyClassifier

In [21]:
classifier = DummyClassifier(strategy="stratified")

In [22]:
classifier.fit(
    posters,
    genre_labels
)

DummyClassifier(strategy='stratified')

In [23]:
Y_pred = pandas.DataFrame(
    classifier.predict(posters),
    columns=genre_labels.columns
)
Y_pred

,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,Genre_Family,Genre_Fantasy,Genre_History,Genre_Horror,Genre_Music,Genre_Mystery,Genre_Romance,Genre_Science Fiction,Genre_TV Movie,Genre_Thriller,Genre_War,Genre_Western
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11816,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11817,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11818,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
11819,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [24]:
Y_pred.sum(axis="columns").value_counts()

2    3555
3    2700
1    2583
4    1409
0     877
5     515
6     141
7      38
8       3
dtype: int64

## Evaluation

In [25]:
scoring = make_scorer(f1_score, greater_is_better=True, average="samples")
cv = 5

In [26]:
cross_val_score(
    DummyClassifier(strategy="uniform"),
    posters,
    genre_labels,
    scoring=scoring,
    cv=cv,
).mean()

0.18571425309291636

In [27]:
cross_val_score(
    DummyClassifier(strategy="stratified"),
    posters,
    genre_labels,
    scoring=scoring,
    cv=cv,
).mean()

0.20197109748032144

In [28]:
cross_val_score(
    DummyClassifier(strategy="most_frequent"),
    posters,
    genre_labels,
    scoring=scoring,
    cv=cv,
).mean()

0.0